In [1]:
from typing import TYPE_CHECKING # 类型检查

from transformers.utils import _LazyModule
from transformers.utils.import_utils import define_import_structure

In [2]:
define_import_structure

<function transformers.utils.import_utils.define_import_structure(module_path: str) -> Dict[FrozenSet[str], Dict[str, Set[str]]]>

In [3]:
TYPE_CHECKING

False

In [ ]:
if TYPE_CHECKING: # 静态检查工具检测时
    from transformers.models.bart.configuration_bart import *
    from transformers.models.bart.modeling_bart import *
    from transformers.models.bart.modeling_flax_bart import *
    from transformers.models.bart.modeling_tf_bart import *
    from transformers.models.bart.tokenization_bart import *
    from transformers.models.bart.tokenization_bart_fast import *
else: # 否则,懒加载
    import sys
    _file = globals()["__file__"]
    sys.modules[__name__] = _LazyModule(__name__, _file, define_import_structure(_file), module_spec=__spec__)

In [5]:
import warnings  # 用于显示警告信息（如弃用警告）
from collections import OrderedDict # 保持字典中键值对的顺序（在构建有序模型配置等场景下常用）
 # 类型提示：Any 表示任意类型，Mapping 表示映射类型，Optional 表示可选（可以是某类型或 None）
from typing import Any, Mapping, Optional
# HuggingFace 的预训练分词器基类，用于文本与 token 之间的转换
from transformers import PreTrainedTokenizer
# 模型配置类的基类，处理模型的参数定义和序列化
from transformers.configuration_utils import PretrainedConfig
# ONNX 导出相关的配置类：
# OnnxConfig：基本导出配置类
# OnnxConfigWithPast：支持 past key-value（通常用于自回归模型）
# OnnxSeq2SeqConfigWithPast：用于支持 encoder-decoder 架构导出，且带有 past 支持
from transformers.onnx import OnnxConfig, OnnxConfigWithPast, OnnxSeq2SeqConfigWithPast
# 工具函数，用于计算某一维的有效大小（可能用于 batch size 或 sequence length 推断）
from transformers.onnx.utils import compute_effective_axis_dimension
# TensorType：用于指明输入的张量类型（如 PyTorch、TensorFlow）
# is_torch_available：判断是否安装了 PyTorch
# logging：Transformers 的日志记录工具（比标准 logging 更一致）
from transformers.utils import TensorType, is_torch_available, logging

In [6]:
is_torch_available()

True

In [7]:
logger = logging.get_logger(__name__) # 日志对象

In [ ]:
# 推理时产生的 past_key_values 不参与模型最终输出的字典中计算 loss 或评估结果，也就是说它是中间
# 缓存结果，用于加速自回归生成的，不用于模型最终输出的结果比较或后处理阶段。
# 我们来精要解释：
#  keys_to_ignore_at_inference 是什么？
# 这是 Hugging Face 的 PretrainedConfig 提供的机制，指示在 model.generate() 或类似推理过程中
# ，模型返回的 output 字典中，这些 key 会被自动忽略，常用于评估、序列解码等场景。
# past_key_values 是什么？
# 这是 Transformer 解码器缓存的 attention key 和 value，用于加速自回归生成。在生成第 t 步时，
# 只需计算当前 token 的注意力，而不是每次都重复前面的 token。
#  为什么要忽略？
# 在推理时它是为了效率存在的，不参与模型主任务（如文本分类、问答等）中的最终输出。指定它为 
# keys_to_ignore_at_inference 的目的：
# 防止影响评估指标或日志记录
# 避免与训练阶段输出做比较时引入多余字段
# ✅ 总结一句话：
# "past_key_values" 被加入 keys_to_ignore_at_inference 表示：它是模型推理时的中间缓存，
# 参与生成，不参与最终输出或结果计算。

In [8]:
# BART 模型的配置类，用于定义模型结构参数，实例化配置后可用于初始化 BART 模型。
# 默认参数对应 [facebook/bart-large] 架构。
class BartConfig(PretrainedConfig):
    
    model_type = "bart" # 指定模型类型
    keys_to_ignore_at_inference = ["past_key_values"]  # 推理时忽略的键
    # 属性映射:外部统一接口映射到内部参数名
    attribute_map = {"num_attention_heads": "encoder_attention_heads", "hidden_size": "d_model"}

    def __init__(
        self,
        vocab_size=50265, # 词表大小
        max_position_embeddings=1024,  # 最大支持的序列长度
        encoder_layers=12,  # 编码器层数
        encoder_ffn_dim=4096,  # 编码器前馈层维度
        encoder_attention_heads=16,  # 编码器 attention 头数
        decoder_layers=12, # 解码器层数
        decoder_ffn_dim=4096,  # 解码器前馈层维度
        decoder_attention_heads=16,  # 解码器 attention 头数
        encoder_layerdrop=0.0,  # 编码器的 LayerDrop 概率
        decoder_layerdrop=0.0,  # 解码器的 LayerDrop 概率
        activation_function="gelu",  # 激活函数，支持 gelu/relu/silu/gelu_new
        d_model=1024,  # 模型隐藏维度，也是 embedding 和 attention 输出的维度
        dropout=0.1,  # 全连接层 dropout 概率
        attention_dropout=0.0,   # attention score 的 dropout
        activation_dropout=0.0,  # 激活函数之后的 dropout
        init_std=0.02,  # 权重初始化标准差（截断正态分布）
        classifier_dropout=0.0,  # 分类头 dropout（用于下游任务）
        scale_embedding=False, # 是否对 embedding 缩放（除以 sqrt(d_model)）
        use_cache=True, # 推理时是否缓存 KV，用于加速自回归生成
        num_labels=3, # 用于分类任务的 label 数（例如 BartForSequenceClassification）
        pad_token_id=1, # pad token id
        bos_token_id=0, # 起始 token id
        eos_token_id=2,  # 结束 token id
        is_encoder_decoder=True,  # 指明该模型为 encoder-decoder 架构
        decoder_start_token_id=2,   # 解码器开始 token
        forced_eos_token_id=2,  # 强制生成结束时最后一个 token
        **kwargs,
    ):
        # 保存参数为实例变量
        self.vocab_size = vocab_size
        self.max_position_embeddings = max_position_embeddings
        self.d_model = d_model
        self.encoder_ffn_dim = encoder_ffn_dim
        self.encoder_layers = encoder_layers
        self.encoder_attention_heads = encoder_attention_heads
        self.decoder_ffn_dim = decoder_ffn_dim
        self.decoder_layers = decoder_layers
        self.decoder_attention_heads = decoder_attention_heads
        self.dropout = dropout
        self.attention_dropout = attention_dropout
        self.activation_dropout = activation_dropout
        self.activation_function = activation_function
        self.init_std = init_std
        self.encoder_layerdrop = encoder_layerdrop
        self.decoder_layerdrop = decoder_layerdrop
        self.classifier_dropout = classifier_dropout
        self.use_cache = use_cache
        self.num_hidden_layers = encoder_layers  # 兼容部分接口用法
        self.scale_embedding = scale_embedding  # 如果为 True，比例因子将为 sqrt(d_model)
         # 调用父类构造，传入特殊 token 和结构配置
        super().__init__(
            num_labels=num_labels,
            pad_token_id=pad_token_id,
            bos_token_id=bos_token_id,
            eos_token_id=eos_token_id,
            is_encoder_decoder=is_encoder_decoder,
            decoder_start_token_id=decoder_start_token_id,
            forced_eos_token_id=forced_eos_token_id,
            **kwargs,
        )

        # 兼容旧版 CNN 模型中强制生成 BOS token 的逻辑
        if self.forced_bos_token_id is None and kwargs.get("force_bos_token_to_be_generated", False):
            self.forced_bos_token_id = self.bos_token_id # 设置强制BOS token id
            warnings.warn(
                f"Please make sure the config includes `forced_bos_token_id={self.bos_token_id}` in future versions. "
                "The config can simply be saved and uploaded again to be fixed."
            )

In [9]:
from transformers import BartModel

2025-05-25 01:39:22.661649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748137162.932442      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748137163.008479      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
# Initializing a BART facebook/bart-large style configuration
configuration = BartConfig()

In [11]:
model = BartModel(configuration)
configuration = model.config
# "_attn_implementation_autoset": true 自动设置底层注意力实现方式，通常无须手动控制。
configuration

BartConfig {
  "_attn_implementation_autoset": true,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_type": "bart",
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": false,
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 50265
}

In [ ]:
# ✅ 功能说明汇总：
# 主要用途：用于导出ONNX模型或集成推理框架时明确每个输入张量维度的语义（如 batch、sequence 长度），便于动态维度标记与图优化。
# self.task：控制当前模型是 seq2seq（如BART）还是 causal-lm（如GPT）。
# self.use_past：控制是否使用 KV 缓存（适用于生成时加速解码）。
# fill_with_past_key_values_：负责动态添加各层的 past key/value 输入定义。

In [32]:
class BartOnnxConfig(OnnxSeq2SeqConfigWithPast):
    # 返回模型的输入映射（input name → 维度索引 → 语义名称），用于推理引擎（如 ONNX）进行维度标记。
    # 返回值:一个有序字典，key 是输入名（如 "input_ids"），value 是另一个 dict，映射输入张量的维度
    # （int）到语义名称（str），如 "batch"、"sequence" 等。
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        # 对于 seq2seq 任务
        if self.task in ["default", "seq2seq-lm"]:
            # 定义通用 encoder 部分输入（输入张量的维度语义）
            common_inputs = OrderedDict(
                [
                    ("input_ids", {0: "batch", 1: "encoder_sequence"}),  # 编码器输入id，形状 (batch, seq_len)
                    ("attention_mask", {0: "batch", 1: "encoder_sequence"}),  # 编码器注意力mask，同上
                ]
            )
             # 启用了 past_key_values（KV缓存机制），表示 decoder 使用的是缓存形式
            # decoder_input_ids 不再需要完整序列，只输入最后一个token
            if self.use_past:
                common_inputs["decoder_input_ids"] = {0: "batch"}  # 解码器输入仅为单步 (batch,)
                # 注意力 mask 维度需支持累积长度（past + 当前）0是批次,1是序列轴 掩码输入就是(b,s)
                common_inputs["decoder_attention_mask"] = {0: "batch", 1: "past_decoder_sequence + sequence"}
            else: # 普通 decoder 输入情况，输入完整序列
                common_inputs["decoder_input_ids"] = {0: "batch", 1: "decoder_sequence"}   # (batch, seq_len)
                common_inputs["decoder_attention_mask"] = {0: "batch", 1: "decoder_sequence"}
            # 为 KV 缓存填充历史key/value输入信息（动态添加past_key_values）
            if self.use_past:
                self.fill_with_past_key_values_(common_inputs, direction="inputs")
        # 对于 causal LM 模型，如 GPT、OPT（只含decoder）
        elif self.task == "causal-lm":
            # encoder_sequence 实际表示自回归序列（兼容GPT的输入）
            common_inputs = OrderedDict(
                [
                    ("input_ids", {0: "batch", 1: "encoder_sequence"}),
                    ("attention_mask", {0: "batch", 1: "encoder_sequence"}),
                ]
            )
            if self.use_past: # 提取层数（注意 causal-lm 模型只有decoder部分，所以只需decoder层数）
                num_encoder_layers, _ = self.num_layers
                for i in range(num_encoder_layers):
                    # past key 和 value 的时间维度索引为 2，维度名为 past_sequence + sequence
                    # 说明这两个张量是 (batch, num_heads, total_seq_len, head_dim)
                    # 0是批次,2是序列维度,因为这里的past_key_values一般是(b,h,s,dk) 2才是需要标记的序列轴
                    common_inputs[f"past_key_values.{i}.key"] = {0: "batch", 2: "past_sequence + sequence"}
                    common_inputs[f"past_key_values.{i}.value"] = {0: "batch", 2: "past_sequence + sequence"}
        else: # fallback 默认行为：返回完整decoder序列输入
            common_inputs = OrderedDict(
                [
                    ("input_ids", {0: "batch", 1: "encoder_sequence"}),
                    ("attention_mask", {0: "batch", 1: "encoder_sequence"}),
                    ("decoder_input_ids", {0: "batch", 1: "decoder_sequence"}),
                    ("decoder_attention_mask", {0: "batch", 1: "decoder_sequence"}),
                ]
            )

        return common_inputs
    # 该 @property def outputs(...) 方法定义了模型导出时的 输出张量的命名与维度语义映射，用于 ONNX 结构化定义，使导
    # 出的模型能兼容不同的推理场景（如普通解码和 KV-cache 加速的解码）。
    # 定义 outputs 属性，返回一个映射（如 OrderedDict），描述输出张量的 每个维度的语义含义，供 ONNX 导出使用。
    @property
    def outputs(self) -> Mapping[str, Mapping[int, str]]:
        if self.task in ["default", "seq2seq-lm"]: # 对于 seq2seq-lm 或默认任务：
            common_outputs = super().outputs # 直接调用父类的输出定义，不做额外处理。
        else:  # 对于其他任务（如 causal-lm）
            # 仍调用父类，但此处用的是显示的类名调用方式，兼容某些继承链可能复杂的情况。：
            common_outputs = super(OnnxConfigWithPast, self).outputs
            # 添加支持 KV 缓存的 present key/value 输出：
            # 如果模型支持 use_past，则表示启用 KV 缓存 推理优化。
            if self.use_past:
                num_encoder_layers, _ = self.num_layers
                # 循环每一层，为当前层的 present key 和 present value 添加输出
                # 这些 present.key/value 是下一步推理的 past_key_values 的输入，即缓存的 attention KV。
                # 输出的 present.{i}.key/value 张量 shape 为 (batch_size, num_heads, total_seq_len, head_dim)
                # 因此第 0 轴为 batch，第 2 轴为时间维度，所以标注 {0: "batch", 2: "past_sequence + sequence"}。
                for i in range(num_encoder_layers):
                    common_outputs[f"present.{i}.key"] = {0: "batch", 2: "past_sequence + sequence"}
                    common_outputs[f"present.{i}.value"] = {0: "batch", 2: "past_sequence + sequence"}
        return common_outputs
    # 生成支持 Seq2Seq（Encoder-Decoder）架构 的模型导出所需的 dummy inputs
    def _generate_dummy_inputs_for_default_and_seq2seq_lm(
        self,
        tokenizer: PreTrainedTokenizer, # 用于生成 token 级输入的 tokenizer
        batch_size: int = -1, # dummy 输入的批大小
        seq_length: int = -1, # 输入序列长度
        is_pair: bool = False, # 是否是句对（如问答），用于控制 tokenizer 的输入模式
        framework: Optional[TensorType] = None, # 所用后端框架类型（如 TensorFlow、PyTorch）
    ) -> Mapping[str, Any]:
        # 重用用于 classification/QA 的通用接口，生成 input_ids、attention_mask 等
        encoder_inputs = self._generate_dummy_inputs_for_sequence_classification_and_question_answering(
            tokenizer, batch_size, seq_length, is_pair, framework
        )

        # 解码器输入生成：若启用 KV 缓存，只需一个 token（单步解码），否则用完整序列。
        decoder_seq_length = seq_length if not self.use_past else 1
        # 同样重用 QA/classification 方法生成 decoder 相关输入，并重命名 key 为 decoder_input_ids 等。
        decoder_inputs = self._generate_dummy_inputs_for_sequence_classification_and_question_answering(
            tokenizer, batch_size, decoder_seq_length, is_pair, framework
        )
        
        decoder_inputs = {f"decoder_{name}": tensor for name, tensor in decoder_inputs.items()}
        common_inputs = dict(**encoder_inputs, **decoder_inputs) # 合并 encoder 和 decoder 的输入字典。
        # 构造 past_key_values（若开启 use_past）： KV cache 仅支持在 PyTorch 中构造，若未安装则报错。
        if self.use_past:
            if not is_torch_available():
                raise ValueError("Cannot generate dummy past_keys inputs without PyTorch installed.")
            else:
                import torch
            # 获取编码器和解码器输入的实际维度。
            batch, encoder_seq_length = common_inputs["input_ids"].shape
            decoder_seq_length = common_inputs["decoder_input_ids"].shape[1]
            # 获取 encoder 和 decoder 的 attention 头数，用于确定 KV 缓存的形状。
            num_encoder_attention_heads, num_decoder_attention_heads = self.num_attention_heads
            # 计算 past key/value 的形状：Encoder KV 缓存张量形状：(B, H, L, D)，其中 D = hidden_size // H
            encoder_shape = (
                batch,
                num_encoder_attention_heads,
                encoder_seq_length,
                self._config.hidden_size // num_encoder_attention_heads,
            )
            # 历史序列长度为 decoder_seq_length + 3（特殊token）
            decoder_past_length = decoder_seq_length + 3
            decoder_shape = (
                batch,
                num_decoder_attention_heads,
                decoder_past_length,
                self._config.hidden_size // num_decoder_attention_heads,
            )
            # 拼接解码器 attention_mask 以支持 past：
            # 对解码器 attention_mask 添加 past 部分，使其支持完整长度（past + 当前）
            common_inputs["decoder_attention_mask"] = torch.cat(
                [common_inputs["decoder_attention_mask"], torch.ones(batch, decoder_past_length)], dim=1
            )
            # 构造 past_key_values：
            common_inputs["past_key_values"] = []
            # 初始化 past_key_values 列表 获取 encoder 和 decoder 的层数，并处理它们层数不一致的情况
            num_encoder_layers, num_decoder_layers = self.num_layers
            min_num_layers = min(num_encoder_layers, num_decoder_layers)
            max_num_layers = max(num_encoder_layers, num_decoder_layers) - min_num_layers
            remaining_side_name = "encoder" if num_encoder_layers > num_decoder_layers else "decoder"
            # 构造（最小层数）内的4元组 past：
            # 每一层有 4 个 tensor：decoder 的 key/value 和 encoder 的 key/value
            for _ in range(min_num_layers):
                common_inputs["past_key_values"].append(
                    (
                        torch.zeros(decoder_shape), # decoder.key
                        torch.zeros(decoder_shape), # decoder.value
                        torch.zeros(encoder_shape), # encoder.key
                        torch.zeros(encoder_shape),  # encoder.value
                    )
                )
            # 构造（剩余层）内的2元组 past：
            # 如果 encoder 和 decoder 层数不等，需要补齐多出的层，仅有 key/value 对，不再区分 encoder/decoder。
            shape = encoder_shape if remaining_side_name == "encoder" else decoder_shape
            for _ in range(min_num_layers, max_num_layers):
                common_inputs["past_key_values"].append((torch.zeros(shape), torch.zeros(shape)))
        # 最终返回结构：
        # 包含 input_ids、attention_mask、decoder_input_ids、decoder_attention_mask
        # 若启用 use_past，还包含 past_key_values（一个 list，每层一个 tuple）
        return common_inputs

    def _generate_dummy_inputs_for_causal_lm(
        self,
        tokenizer: PreTrainedTokenizer,
        batch_size: int = -1,
        seq_length: int = -1,
        is_pair: bool = False,
        framework: Optional[TensorType] = None,
    ) -> Mapping[str, Any]:
        # 通用方法：复用用于分类/问答任务的 dummy 输入生成逻辑
        # 主要生成 input_ids、attention_mask 等基础输入
        common_inputs = self._generate_dummy_inputs_for_sequence_classification_and_question_answering(
            tokenizer, batch_size, seq_length, is_pair, framework
        )
        # 若模型支持 KV 缓存（use_past=True），则构造 past_key_values 和扩展 attention_mask
        if self.use_past:
            # past_key_values 构造仅支持 PyTorch 后端
            if not is_torch_available():
                raise ValueError("Cannot generate dummy past_keys inputs without PyTorch installed.")
            else:
                import torch
             # 获取当前输入的 batch 和序列长度
            batch, seqlen = common_inputs["input_ids"].shape
            # 假设历史缓存长度为当前长度 + 2，用于测试 past key/value 的拼接兼容性
            past_key_values_length = seqlen + 2
            # 取出 encoder 层数和注意力头数（GPT 等 causal LM 通常只有 encoder）
            num_encoder_layers, _ = self.num_layers
            num_encoder_attention_heads, _ = self.num_attention_heads
            # 构造 past_key/past_value 的张量形状：(batch, num_heads, past_len, head_dim)
            past_shape = (
                batch,
                num_encoder_attention_heads,
                past_key_values_length,
                self._config.hidden_size // num_encoder_attention_heads,
            )
            # 获取 attention_mask 的数据类型，保证拼接后类型一致
            mask_dtype = common_inputs["attention_mask"].dtype
            # 拼接 attention_mask，扩展 past 部分为 1，表示 "已存在" 的 token
            common_inputs["attention_mask"] = torch.cat(
                [common_inputs["attention_mask"], torch.ones(batch, past_key_values_length, dtype=mask_dtype)], dim=1
            )
            # 构造 past_key_values，为每一层提供 (key, value) 的零张量
            # 注意：causal LM 中只构造 (key, value)，没有 encoder 部分
            common_inputs["past_key_values"] = [
                (torch.zeros(past_shape), torch.zeros(past_shape)) for _ in range(num_encoder_layers)
            ]
         # 返回包含 input_ids、attention_mask（以及可选的 past_key_values）的输入字典
        return common_inputs
    # 此函数为生成伪输入数据的核心方法，供 sequence classification 和 QA 模型导出 ONNX 使用
    # 注：并未使用 super().generate_dummy_inputs 是为了增强代码清晰度和自定义灵活性
    # 若 batch_size 为 -1，表示采用动态轴，为避免 ONNX 导出中某些静态优化行为，
    # 则将其替换为固定值（如 2），确保推理兼容性和导出稳定性
    def _generate_dummy_inputs_for_sequence_classification_and_question_answering(
        self,
        tokenizer: PreTrainedTokenizer,
        batch_size: int = -1,
        seq_length: int = -1,
        is_pair: bool = False,
        framework: Optional[TensorType] = None,
    ) -> Mapping[str, Any]:
        
        batch_size = compute_effective_axis_dimension(
            batch_size,
            fixed_dimension=OnnxConfig.default_fixed_batch, # 设置默认批次大小为2
            num_token_to_add=0  # 无需添加特殊 token
        )

        # 如果是动态轴（-1），我们以 8 个 token 的固定维度进行forward，以避免 ONNX 进行的优化
        # 同理，如果 seq_length 为 -1，则设为固定值（如 8），避免 ONNX 静态路径优化
        token_to_add = tokenizer.num_special_tokens_to_add(is_pair)  # 根据是否是 pair 输入决定是否添加特殊 token
        seq_length = compute_effective_axis_dimension(
            seq_length,
            fixed_dimension=OnnxConfig.default_fixed_sequence, # 设置默认序列长度为8
            num_token_to_add=token_to_add  # 需要预留特殊 token 长度
        )

        # 根据计算批次和序列生成虚拟输入
        # 构造 dummy 输入：使用 unknown token 构造文本串，个数为 batch_size，每条长度为 seq_length
        dummy_input = [" ".join([tokenizer.unk_token]) * seq_length] * batch_size
        # 使用 tokenizer 编码输入，返回张量形式的输入字典（如 input_ids, attention_mask）
        # return_tensors 指定输出格式（如 "pt" or "np"）
        common_inputs = dict(tokenizer(dummy_input, return_tensors=framework))
        return common_inputs  # 返回输入张量字典，供后续模型或 ONNX trace 使用
    # 生成虚拟输入,分支判断并调用上面定义的私有的方法
    def generate_dummy_inputs(
        self,
        tokenizer: PreTrainedTokenizer,
        batch_size: int = -1,
        seq_length: int = -1,
        is_pair: bool = False,
        framework: Optional[TensorType] = None,
    ) -> Mapping[str, Any]:
        if self.task in ["default", "seq2seq-lm"]: # 如果任务类型是"default", "seq2seq-lm"
            common_inputs = self._generate_dummy_inputs_for_default_and_seq2seq_lm(
                tokenizer, batch_size=batch_size, seq_length=seq_length, is_pair=is_pair, framework=framework
            )

        elif self.task == "causal-lm":
            common_inputs = self._generate_dummy_inputs_for_causal_lm(
                tokenizer, batch_size=batch_size, seq_length=seq_length, is_pair=is_pair, framework=framework
            )
        else:
            common_inputs = self._generate_dummy_inputs_for_sequence_classification_and_question_answering(
                tokenizer, batch_size=batch_size, seq_length=seq_length, is_pair=is_pair, framework=framework
            )

        return common_inputs
    # 该函数用于处理 past_key_values 的展开（flatten）操作，
    # 以便在 ONNX 导出过程中将嵌套结构（如列表中的多个张量）转换为扁平的字典形式，
    # 例如：将 [(k0, v0), (k1, v1), ...] 展开为 {"past_key_values.0.key": ..., "past_key_values.0.value": ..., ...}
    def _flatten_past_key_values_(self, flattened_output, name, idx, t):
        if self.task in ["default", "seq2seq-lm"]: 
            # 对于 "default" 和 "seq2seq-lm" 类型任务，调用父类默认实现，
            # 通常适用于 encoder-decoder 模型如 BART、T5 等
            flattened_output = super()._flatten_past_key_values_(flattened_output, name, idx, t)
        else:
            # 对于其他任务（如 causal-lm，纯 decoder 架构），显式调用 OnnxSeq2SeqConfigWithPast 的父类实现
            # 防止因多重继承或方法重写引发冲突或歧义
            flattened_output = super(OnnxSeq2SeqConfigWithPast, self)._flatten_past_key_values_(
                flattened_output, name, idx, t
            )

In [15]:
onnxConfig=BartOnnxConfig(configuration)

In [27]:
onnxConfig.task

'default'

In [17]:
onnxConfig.inputs

OrderedDict([('input_ids', {0: 'batch', 1: 'encoder_sequence'}),
             ('attention_mask', {0: 'batch', 1: 'encoder_sequence'}),
             ('decoder_input_ids', {0: 'batch', 1: 'decoder_sequence'}),
             ('decoder_attention_mask', {0: 'batch', 1: 'decoder_sequence'})])

In [19]:
onnxConfig.outputs

OrderedDict([('last_hidden_state', {0: 'batch', 1: 'decoder_sequence'})])

In [33]:
onnxConfig.use_past

False

In [22]:
print(OnnxConfig.default_fixed_batch,OnnxConfig.default_fixed_sequence)

2 8


In [25]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

In [26]:
onnxConfig.generate_dummy_inputs(tokenizer)

{'input_ids': [[0, 3, 3, 3, 3, 3, 3, 2], [0, 3, 3, 3, 3, 3, 3, 2]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]],
 'decoder_input_ids': [[0, 3, 3, 3, 3, 3, 3, 2], [0, 3, 3, 3, 3, 3, 3, 2]],
 'decoder_attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
__all__ = ["BartConfig", "BartOnnxConfig"]